## Preliminary preparations

In [1]:
from datetime import datetime
import os
from pathlib import Path

import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
import warnings

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
pd.set_option('display.max_rows', 100)

In [2]:
logfile = "../logs/512G/2025-01-08-20-00-20-mainnet-beta-512G.log"
csv = "../logs/512G/tps_512gb.csv"

## Preparing Data Source

In [3]:
data = []

with open(logfile, 'r') as file:
    for line in file:
        if "datapoint: replay-slot-stats" in line:

            timestamp_match = re.search(r"\[(.*?)Z", line)
            transactions_match = re.search(r"total_transactions=(\d+)i", line)
            execute_us_match = re.search(r"execute_us=(\d+)i", line)
            
            if timestamp_match and transactions_match and execute_us_match:
                timestamp = timestamp_match.group(1)
                total_transactions_executed = int(transactions_match.group(1))
                execute_us = int(execute_us_match.group(1))

                data.append([timestamp, total_transactions_executed, execute_us])

df = pd.DataFrame(data, columns=['timestamp', 'total_transactions_executed', 'execute_us'])
df

,timestamp,total_transactions_executed,execute_us
0,2025-01-08T20:37:11.828440284,1547,734800
1,2025-01-08T20:37:12.340340319,1557,309182
2,2025-01-08T20:37:12.986499485,1672,736146
3,2025-01-08T20:37:13.833668449,2263,534700
4,2025-01-08T20:37:14.110291708,1664,212916
...,...,...,...
101620,2025-01-09T07:57:13.010411902,2343,541054
101621,2025-01-09T07:57:13.275366695,1750,450590
101622,2025-01-09T07:57:13.664051354,1688,453770
101623,2025-01-09T07:57:14.022117023,1461,407793


In [4]:
def convert_to_unixtime(timestamp):
    base_time, microseconds = timestamp.split('.')
    microseconds = microseconds[:6]
    dt = datetime.strptime(base_time, '%Y-%m-%dT%H:%M:%S')
    return int(dt.timestamp() * 1_000_000) + int(microseconds)

In [5]:
df['end'] = df['timestamp'].apply(convert_to_unixtime)
df['start'] = df['end'] - df['execute_us']
df['tps'] = df['total_transactions_executed'] * 1000000 / df['execute_us']
df

,timestamp,total_transactions_executed,execute_us,end,start,tps
0,2025-01-08T20:37:11.828440284,1547,734800,1736397431828440,1736397431093640,2105.334785
1,2025-01-08T20:37:12.340340319,1557,309182,1736397432340340,1736397432031158,5035.868841
2,2025-01-08T20:37:12.986499485,1672,736146,1736397432986499,1736397432250353,2271.288576
3,2025-01-08T20:37:13.833668449,2263,534700,1736397433833668,1736397433298968,4232.279783
4,2025-01-08T20:37:14.110291708,1664,212916,1736397434110291,1736397433897375,7815.288658
...,...,...,...,...,...,...
101620,2025-01-09T07:57:13.010411902,2343,541054,1736438233010411,1736438232469357,4330.436518
101621,2025-01-09T07:57:13.275366695,1750,450590,1736438233275366,1736438232824776,3883.796800
101622,2025-01-09T07:57:13.664051354,1688,453770,1736438233664051,1736438233210281,3719.946228
101623,2025-01-09T07:57:14.022117023,1461,407793,1736438234022117,1736438233614324,3582.700046


In [6]:
start_time = df.start.min() // 1000000 * 1000000
end_time = (df.start.max() + 1000000) // 1000000 * 1000000
time_step = 100000 # 10,000 microsends = .1 second

time_series = pd.DataFrame({
    't': np.arange(start_time, end_time + time_step, time_step)
})

def aggregate_tps(row, df):
    mask = (df['start'] <= row['t']) & (df['end'] >= row['t'])
    return df.loc[mask, 'tps'].sum()

time_series['sum_tps'] = time_series.apply(aggregate_tps, axis=1, df=df)
# window=10 means creating a moving average with the past 1 second's data. If you change it to 100, it would be the past 10 seconds
time_series['moving_average'] = time_series['sum_tps'].rolling(window=50, min_periods=1).mean()
time_series.to_csv(csv, index=False)


In [7]:
# Calculate the maximum TPS
max_tps = time_series['sum_tps'].max()

# Calculate the overall average TPS
average_tps = time_series['sum_tps'].mean()

# Print the results
print(f"Maximum TPS: {max_tps:,.2f}")
print(f"Average TPS: {average_tps:,.2f}")

# Convert the max TPS x-coordinate to the corresponding time
max_tps_time = time_series.loc[time_series['sum_tps'].idxmax(), 't']

# Calculate the total duration spent with 0 TPS
time_with_zero_tps = time_series[time_series['sum_tps'] == 0]
total_time_zero_tps = len(time_with_zero_tps) * time_step / 1_000_000  # Convert microseconds to seconds
total_time_zero_tps_hours = total_time_zero_tps / 3600  # Convert seconds to hours

print(f"Total time spent with 0 TPS: {total_time_zero_tps:.2f} seconds ({total_time_zero_tps_hours:.2f} hours)")

Maximum TPS: 122,269.14
Average TPS: 4,642.67
Total time spent with 0 TPS: 13492.90 seconds (3.75 hours)


In [8]:
# Display every 10 seconds
filtered_time_series = time_series[time_series['t'] % 10000000 == 0]
# from micro sec to sec
filtered_time_series['t'] = np.floor(filtered_time_series['t'] / 1000000)

filtered_time_series

,t,sum_tps,moving_average
90,1.736397e+09,3531.494193,6348.988616
190,1.736397e+09,0.000000,6027.233735
290,1.736397e+09,4491.281294,8507.514161
390,1.736397e+09,11887.242006,8596.958368
490,1.736397e+09,4932.454640,5033.637455
...,...,...,...
407590,1.736438e+09,2574.748804,4573.366895
407690,1.736438e+09,0.000000,3198.487449
407790,1.736438e+09,6593.324653,3313.604659
407890,1.736438e+09,0.000000,2842.578419


In [9]:
# Calculate time elapsed in seconds from the start of the log
start_time = filtered_time_series['t'].min()  # First timestamp
filtered_time_series['elapsed_time'] = filtered_time_series['t'] - start_time
filtered_time_series['elapsed_time_hours'] = filtered_time_series['elapsed_time'] / 3600

# Convert elapsed time to HH:MM:SS format
filtered_time_series['elapsed_time_formatted'] = filtered_time_series['elapsed_time'].apply(
    lambda x: f"{int(x // 3600):02}:{int((x % 3600) // 60):02}:{int(x % 60):02}"
)

In [10]:
# Determine the range for the x-axis
x_min = filtered_time_series['elapsed_time_hours'].min() - 1  # One hour before the first data point
x_max = filtered_time_series['elapsed_time_hours'].max() + 1  # One hour after the last data point

fig = go.Figure()

fig.add_trace(go.Scatter(x=filtered_time_series['elapsed_time_hours'], y=filtered_time_series['sum_tps'], mode='markers', name='TPS', marker=dict(size=2)))
fig.add_trace(go.Scatter(x=filtered_time_series['elapsed_time_hours'], y=filtered_time_series['moving_average'], mode='lines', name='TPS Moving Average(1sec)', line=dict(width=0.5)))

# Add horizontal lines for max and average TPS
fig.add_hline(y=average_tps, line_dash="dot", line_color="green", annotation_text=f"Average TPS: {average_tps:.2f}", annotation_position="bottom right")
fig.add_hline(y=max_tps, line_dash="dash", line_color="red", annotation_text=f"Max TPS: {max_tps:.2f}", annotation_position="top right")

fig.update_layout(
    title=f"TPS Over Time | Total Time Spent with 0 TPS: {total_time_zero_tps:.2f} seconds ({total_time_zero_tps_hours:.2f} hours) | Max TPS: {max_tps:,.2f}",
    xaxis_title="Time Elapsed (hours)",
    yaxis_title="TPS",
    xaxis=dict(
        tickmode='linear',
        dtick=1,
        tickformat=".1f",
        range=[x_min, x_max]
    ),
    yaxis=dict(range=[0, 15000])
)


fig.show()


# ## Notes
# - If all data points are plotted on the graph, the points and lines become indistinguishable, so I plot them every 10 seconds.
# - TPS values above 15000 are excluded as outliers.
